In [ ]:
# Directory 
input_path = os.environ.get('input_path')
fig_output = os.environ.get('fig_output')
tab_output = os.environ.get('tab_output')

# Queries config
project_id_bq = os.environ.get('project_id_bq')
run_query = os.environ.get('run_query')

In [ ]:
# Total of municipalities from our commitment data in relation with Siconfi data

def run_query_and_save_results():

    query = '''
    WITH commitment AS (
      SELECT
        ano,
        sigla_uf,
        COUNT (DISTINCT id_municipio) AS municipalities_tce,
      FROM `basedosdados.world_wb_mides.empenho`
      GROUP BY 1,2
    ),
      siconfi AS (
      SELECT
      ano,
      sigla_uf,
      COUNT (DISTINCT id_municipio) AS municipalities_siconfi
      FROM `basedosdados.br_me_siconfi.municipio_despesas_orcamentarias`
      WHERE sigla_uf IN ('CE', 'MG', 'PB', 'PE', 'PR', 'RS', 'SP') AND ano >= 2008 AND estagio_bd = 'Despesas Empenhadas' AND conta_bd = 'Despesas Orçamentárias'
      GROUP BY 1,2
    )

    SELECT
      s.ano AS year_siconfi,
      e.ano AS year_tce,
      s.sigla_uf AS state_siconfi,
      e.sigla_uf AS state_tce,
      municipalities_tce,
      municipalities_siconfi
    FROM commitment e
    FULL OUTER JOIN siconfi s ON e.ano=s.ano AND e.sigla_uf=s.sigla_uf
    '''

    commitment_siconfi = bd.read_sql(query, billing_project_id=project_id_bq)
    
    commitment_siconfi.to_csv(os.path.join(input_path,'data_commitment_siconfi.csv'), index=False, na_rep='', float_format='%.2f')

if __name__ == '__main__':
    if run_query == 'True':
        run_query_and_save_results()

In [ ]:
# Total of municipalities from our verification data in relation with Siconfi data

def run_query_and_save_results():

    query = '''
    WITH verification AS (
      SELECT
        ano,
        sigla_uf,
        COUNT (DISTINCT id_municipio) AS municipalities_tce,
      FROM `basedosdados.world_wb_mides.liquidacao`
      GROUP BY 1,2
    ),
      siconfi AS (
      SELECT
      ano,
      sigla_uf,
      COUNT (DISTINCT id_municipio) AS municipalities_siconfi
      FROM `basedosdados.br_me_siconfi.municipio_despesas_orcamentarias`
      WHERE sigla_uf IN ('CE', 'MG', 'PB', 'PE', 'PR', 'RS', 'SP') AND ano >= 2008 AND estagio_bd = 'Despesas Liquidadas' AND conta_bd = 'Despesas Orçamentárias'
      GROUP BY 1,2
    )

    SELECT
      s.ano AS year_siconfi,
      e.ano AS year_tce,
      s.sigla_uf AS state_siconfi,
      e.sigla_uf AS state_tce,
      municipalities_tce,
      municipalities_siconfi
    FROM verification e
    FULL OUTER JOIN siconfi s ON e.ano=s.ano AND e.sigla_uf=s.sigla_uf
    '''

    verification_siconfi = bd.read_sql(query, billing_project_id=project_id_bq)

    verification_siconfi.to_csv(os.path.join(input_path,'data_verification_siconfi.csv'), index=False, na_rep='', float_format='%.2f')

if __name__ == '__main__':
    if run_query == 'True':
        run_query_and_save_results()

In [ ]:
# Total of municipalities from our commitment data in relation with Siconfi data

def run_query_and_save_results():

    query = '''
    WITH payment AS (
      SELECT
        ano,
        sigla_uf,
        COUNT (DISTINCT id_municipio) AS municipalities_tce,
      FROM `basedosdados.world_wb_mides.pagamento`
      GROUP BY 1,2
    ),
      siconfi AS (
      SELECT
      ano,
      sigla_uf,
      COUNT (DISTINCT id_municipio) AS municipalities_siconfi
      FROM `basedosdados.br_me_siconfi.municipio_despesas_orcamentarias`
      WHERE sigla_uf IN ('CE', 'MG', 'PB', 'PE', 'PR', 'RS', 'SP') AND ano >= 2008 AND estagio_bd = 'Despesas Pagas' AND conta_bd = 'Despesas Orçamentárias'
      GROUP BY 1,2
    )

    SELECT
      s.ano AS year_siconfi,
      e.ano AS year_tce,
      s.sigla_uf AS state_siconfi,
      e.sigla_uf AS state_tce,
      municipalities_tce,
      municipalities_siconfi
    FROM payment e
    FULL OUTER JOIN siconfi s ON e.ano=s.ano AND e.sigla_uf=s.sigla_uf
    '''

    payment_siconfi = bd.read_sql(query, billing_project_id=project_id_bq)

    payment_siconfi.to_csv(os.path.join(input_path,'data_payment_siconfi.csv'), index=False, na_rep='', float_format='%.2f')

if __name__ == '__main__':
    if run_query == 'True':
        run_query_and_save_results()

In [ ]:
# Total of municipalities from our commitment data in relation with Siconfi data

commitment_siconfi = pd.read_csv(os.path.join(input_path, 'data_commitment_siconfi.csv'))

commitment_siconfi['year_tce'] = commitment_siconfi['year_tce'].replace(np.nan, 0).astype(int)
commitment_siconfi = commitment_siconfi.sort_values(by=['state_siconfi', 'state_tce', 'year_siconfi', 'year_tce'])

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3,2, figsize=(12,8))
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=['#1a476f', '#c10534'])

commitment_siconfi.query('state_tce == "CE"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax1, marker='.', legend=None)
commitment_siconfi.query('state_tce == "MG"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax2, marker='.', legend=None)
commitment_siconfi.query('state_tce == "PB" & year_tce > 2008').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax3, marker='.', legend=None)
commitment_siconfi.query('state_tce == "PR"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax4, marker='.', legend=None)
commitment_siconfi.query('state_tce == "RS" & year_tce > 2009').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax5, marker='.', legend=None)
commitment_siconfi.query('state_tce == "SP"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax6, marker='.', legend=None)

ax1.set_title("CE")
ax1.set_ylabel('Municipalities')
ax1.set_xlabel(None)
ax1.set_xticks(np.arange(2009,2022,2))
ax2.set_title("MG")
ax2.set_xlabel(None)
ax3.set_title("PB")
ax3.set_yticks([210,213,216,219,221,223])
ax3.set_ylabel('Municipalities')
ax3.set_xlabel(None)
ax3.set_xticks(np.arange(2009,2021,2))
ax4.set_title("PR")
ax4.set_xlabel(None)
ax5.set_title("RS")
ax5.set_ylabel('Municipalities')
ax5.set_xlabel('Year')
ax6.set_title("SP")
ax6.set_xlabel('Year')


plt.tight_layout()
plt.legend(['Municipalities from our data', 'Municipalities from Siconfi'], bbox_to_anchor =(-0.1,-0.5), loc='lower center', ncol=2)
plt.savefig(os.path.join(fig_output, 'total_municipalities_commitment.pdf'))

In [ ]:
# Total of municipalities from our verification data in relation with Siconfi data

verification_siconfi = pd.read_csv(os.path.join(input_path, 'data_verification_siconfi.csv'))

verification_siconfi['year_tce'] = verification_siconfi['year_tce'].replace(np.nan, 0).astype(int)
verification_siconfi = verification_siconfi.sort_values(by=['state_siconfi', 'state_tce', 'year_siconfi', 'year_tce'])

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3,2, figsize=(12,8))
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=['#1a476f', '#c10534'])

verification_siconfi.query('state_tce == "CE"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax1, marker='.', legend=None)
verification_siconfi.query('state_tce == "MG"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax2, marker='.', legend=None)
verification_siconfi.query('state_tce == "PB" & year_tce > 2008').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax3, marker='.', legend=None)
verification_siconfi.query('state_tce == "PR"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax4, marker='.', legend=None)
verification_siconfi.query('state_tce == "RS" & year_tce > 2009').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax5, marker='.', legend=None)
verification_siconfi['year_siconfi'] = np.where(verification_siconfi.year_tce == 2008, 2008, verification_siconfi.year_siconfi)
verification_siconfi['state_siconfi'] = np.where(verification_siconfi.year_tce == 2008, 'SP', verification_siconfi.state_siconfi)
verification_siconfi.query('state_tce == "SP"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax6, marker='.', legend=None)

ax1.set_title("CE")
ax1.set_ylabel('Municipalities')
ax1.set_xlabel(None)
ax1.set_xticks(np.arange(2009,2022,2))
ax2.set_title("MG")
ax2.set_xlabel(None)
ax3.set_title("PB")
ax3.set_yticks([210,213,216,219,221,223])
ax3.set_ylabel('Municipalities')
ax3.set_xlabel(None)
ax3.set_xticks(np.arange(2009,2021,2))
ax4.set_title("PR")
ax4.set_xlabel(None)
ax5.set_title("RS")
ax5.set_ylabel('Municipalities')
ax5.set_xlabel('Year')
ax6.set_title("SP")
ax6.set_xlabel('Year')


plt.tight_layout()
plt.legend(['Municipalities from our data', 'Municipalities from Siconfi'], bbox_to_anchor =(-0.1,-0.5), loc='lower center', ncol=2)
plt.savefig(os.path.join(fig_output, 'total_municipalities_verification.pdf'))

In [ ]:
# Total of municipalities from our commitment data in relation with Siconfi data

payment_siconfi = pd.read_csv(os.path.join(input_path, 'data_payment_siconfi.csv'))

payment_siconfi['year_tce'] = payment_siconfi['year_tce'].replace(np.nan, 0).astype(int)
payment_siconfi = payment_siconfi.sort_values(by=['state_siconfi', 'state_tce', 'year_siconfi', 'year_tce'])

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3,2, figsize=(12,8))
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=['#1a476f', '#c10534'])

payment_siconfi.query('state_tce == "CE"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax1, marker='.', legend=None)
payment_siconfi.query('state_tce == "MG"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax2, marker='.', legend=None)
payment_siconfi.query('state_tce == "PB" & year_tce > 2008').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax3, marker='.', legend=None)
payment_siconfi.query('state_tce == "PR"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax4, marker='.', legend=None)
payment_siconfi.query('state_tce == "RS" & year_tce > 2009').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax5, marker='.', legend=None)
payment_siconfi.query('state_tce == "SP"').plot(x="year_tce", y=["municipalities_tce", "municipalities_siconfi"], kind="line", ax=ax6, marker='.', legend=None)

ax1.set_title("CE")
ax1.set_ylabel('Municipalities')
ax1.set_xlabel(None)
ax1.set_xticks(np.arange(2009,2022,2))
ax2.set_title("MG")
ax2.set_xlabel(None)
ax3.set_title("PB")
ax3.set_yticks([210,213,216,219,221,223])
ax3.set_ylabel('Municipalities')
ax3.set_xlabel(None)
ax3.set_xticks(np.arange(2009,2021,2))
ax4.set_title("PR")
ax4.set_xlabel(None)
ax5.set_title("RS")
ax5.set_ylabel('Municipalities')
ax5.set_xlabel('Year')
ax6.set_title("SP")
ax6.set_xlabel('Year')


plt.tight_layout()
plt.legend(['Municipalities from our data', 'Municipalities from Siconfi'], bbox_to_anchor =(-0.1,-0.5), loc='lower center', ncol=2)
plt.savefig(os.path.join(fig_output, 'total_municipalities_payment.pdf'))